Part 1: Retrieval-Augmented Generation (RAG) Model for QA Bot
 Problem Statement:
 Develop a Retrieval-Augmented Generation (RAG) model for a Question Answering (QA)
 bot for a business. Use a vector database like Pinecone DB and a generative model like
 Cohere API (or any other available alternative). The QA bot should be able to retrieve
 relevant information from a dataset and generate coherent answers.
 Task Requirements:
 1. Implement a RAG-based model that can handle questions related to a provided
 document or dataset.
 2. Use a vector database (such as Pinecone) to store and retrieve document
 embeddings efficiently.
 3. Test the model with several queries and show how well it retrieves and generates
 accurate answers from the document.
 Deliverables:
 ● A Colab notebook demonstrating the entire pipeline, from data loading to question
 answering.
 ● Documentation explaining the model architecture, approach to retrieval, and how
 generative responses are created.
 ● Provide several example queries and the corresponding outputs.


In [1]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF
!pip install sentence-transformers
!pip install pinecone-client
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [4]:
import fitz  # PyMuPDF for PDF text extraction

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

# Load the PDF and extract text
pdf_file = "/content/Black hole article.pdf"
document_text = extract_text_from_pdf(pdf_file)

# Optional: Preview the first 1000 characters of the extracted text
print(document_text[:1000])

See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/1818209
Black Holes : A General Introduction
Article  in  Lecture Notes in Physics · February 1998
DOI: 10.1007/978-3-540-49535-2_1 · Source: arXiv
CITATIONS
24
READS
44,724
1 author:
Jean-Pierre Luminet
Laboratoire d'Astrophysique de Marseille
139 PUBLICATIONS   3,119 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Jean-Pierre Luminet on 18 February 2013.
The user has requested enhancement of the downloaded file.
arXiv:astro-ph/9801252v1  26 Jan 1998
Black Holes: A General Introduction
Jean-Pierre Luminet
Observatoire de Paris-Meudon, D´epartement d’Astrophysique Relativiste et de
Cosmologie, CNRS UPR-176, F-92195 Meudon Cedex, France
Abstract. Our understanding of space and time is probed to its depths by black holes.
These objects, which appear as a natural consequence of general relativity, provide a
powerful analytical tool able to examine macrosco

In [5]:
import nltk
nltk.download('punkt')

# Split the text into sentences for embedding generation
sentences = nltk.sent_tokenize(document_text)
print(f"Number of sentences: {len(sentences)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Number of sentences: 441


In [6]:
from sentence_transformers import SentenceTransformer

# Initialize the model for embedding generation
model = SentenceTransformer('all-mpnet-base-v2')

# Generate embeddings for the sentences
sentence_embeddings = model.encode(sentences)

# Example: Check the embedding for the first sentence
print(f"First sentence: {sentences[0]}")
print(f"First sentence embedding shape: {sentence_embeddings[0].shape}")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

First sentence: See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/1818209
Black Holes : A General Introduction
Article  in  Lecture Notes in Physics · February 1998
DOI: 10.1007/978-3-540-49535-2_1 · Source: arXiv
CITATIONS
24
READS
44,724
1 author:
Jean-Pierre Luminet
Laboratoire d'Astrophysique de Marseille
139 PUBLICATIONS   3,119 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Jean-Pierre Luminet on 18 February 2013.
First sentence embedding shape: (768,)


In [7]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
pc = Pinecone(
    api_key='ce4bd8f2-6ed6-4147-8cf5-84a8f1a0f10a'  #actual API key
)

# Check if the index exists, if not create one
index_name = 'blackhole'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Replace with the actual dimensionality of your embeddings
        metric='euclidean',  # Choose the appropriate metric (euclidean, cosine, dotproduct)
        spec=ServerlessSpec(
            cloud='aws',  # Cloud provider
            region='us-east-1'  # Change the region to 'us-east-1' for the free plan
        )
    )

# Connect to the Pinecone index
index = pc.Index(index_name)

In [8]:
import cohere
import numpy as np

# Initialize Cohere API for generating answers
cohere_client = cohere.Client('WqKIAPOUFp82pRRbxmJKTDuiDpkladUtMrcS3lSE')

# Example query
query = "what is black hole theory"

# Convert the query to an embedding
# Ensure `model.encode` is a valid method; replace with actual model if necessary
query_embedding = model.encode([query])[0].tolist()  # Convert numpy array to list

# Retrieve relevant document sections from Pinecone
result = index.query(vector=query_embedding, top_k=5)

# Extract the document IDs and their corresponding sentences
retrieved_sentences = [sentences[int(match['id'])] for match in result['matches']]

# Use Cohere to generate an answer based on the retrieved text
response = cohere_client.generate(
    prompt=f"Question: {query}\n\nContext: {' '.join(retrieved_sentences)}\n\nAnswer:",
    max_tokens=100
)

# Display the generated answer
print(f"Generated Answer: {response.generations[0].text}")

Generated Answer:  Black hole theory is a scientific theory that explains the phenomena and mechanics of black holes. 

Black holes are regions of space where an enormous amount of mass is packed into a tiny volume. This creates a gravitational pull so strong that not even electromagnetic waves (i.e. light) can escape. 

The theory explains that black holes form when very massive stars collapse at the end of their life cycle. This process can also happen when galaxies merge. What is left behind is an extremely dense object
